In [97]:
import time
import pandas as pd
from selenium import webdriver

In [60]:
driver = webdriver.Chrome("chromedriver.exe")

In [87]:
driver.switch_to.window(driver.window_handles[1])

In [101]:
mail_list = [
    { "std_name" : "", "mail" : "" },
    { "std_name" : "", "mail" : "" }
]

In [ ]:
result_list = []
for mail in mail_list:
    try:
        driver.find_element_by_css_selector("#searchWordTxt").clear()
        driver.find_element_by_css_selector("#searchWordTxt").send_keys(mail["mail"])
    except:
        driver.find_element_by_css_selector("#searchWord").clear()
        driver.find_element_by_css_selector("#searchWord").send_keys(mail["mail"])        
    driver.find_element_by_css_selector("#popWrap > div.popContents.popCorp > div > div.sArea > div.tab_wrap.clfix > div > fieldset > div > div.btnArea.sInput_box > ul > li > span > input[type=button]").click()
    time.sleep(2)
    try:
        result_list.append({ 
            "자녀이름" : mail["std_name"],
            "이름" : driver.find_element_by_css_selector("#popWrap > div.popContents.popCorp > div > div.sResult > div.pInfoView > div > div > div > table > tbody > tr:nth-child(2) > td:nth-child(3) > div > div").text.strip().split(" / ")[0],
            "부서" : driver.find_element_by_css_selector("#popWrap > div.popContents.popCorp > div > div.sResult > div.pInfoView > div > div > div > table > tbody > tr:nth-child(3) > td:nth-child(2) > div > div.tb_text_area > a").text.strip(),
            "전화번호" : driver.find_element_by_css_selector("#popWrap > div.popContents.popCorp > div > div.sResult > div.pInfoView > div > div > table > tbody > tr:nth-child(1) > td > span > div > div.tb_text_area").text.strip(),
            "휴대전화" : driver.find_element_by_css_selector("#popWrap > div.popContents.popCorp > div > div.sResult > div.pInfoView > div > div > table > tbody > tr:nth-child(2) > td:nth-child(2) > div > div.tb_text_area").text.strip(),
            "이메일" : driver.find_element_by_css_selector("#popWrap > div.popContents.popCorp > div > div.sResult > div.pInfoView > div > div > table > tbody > tr:nth-child(2) > td:nth-child(4) > div > div.tb_text_area > a").text.strip(),
        })
    except:
        pass
df = pd.DataFrame(result_list)
df.to_excel("result.xlsx", index=False)
df